In [1]:
# GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"



from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

2025-11-27 13:15:03.923269: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9013011055966822284
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7775453184
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2192724244442996449
physical_device_desc: "device: 0, name: Quadro M4000, pci bus id: 0000:03:00.0, compute capability: 5.2"
xla_global_id: 416903419
]


2025-11-27 13:15:06.561450: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /device:GPU:0 with 7415 MB memory:  -> device: 0, name: Quadro M4000, pci bus id: 0000:03:00.0, compute capability: 5.2


In [2]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, LearningRateScheduler
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras import regularizers

import tensorflow.keras.backend as K

import h5py

from sklearn.model_selection import train_test_split

import os

In [3]:
os.getcwd()

'/echo3/hadjik01/Python'

In [4]:
os.environ

environ{'SHELL': '/bin/bash',
        'CONDA_EXE': '/home/hadjik01/miniforge3/bin/conda',
        '_CE_M': '',
        'LC_ADDRESS': 'de_DE.UTF-8',
        'LC_NAME': 'de_DE.UTF-8',
        'LC_MONETARY': 'de_DE.UTF-8',
        'XML_CATALOG_FILES': 'file:///home/hadjik01/miniforge3/envs/tf/etc/xml/catalog file:///etc/xml/catalog',
        'PWD': '/home/hadjik01',
        'GSETTINGS_SCHEMA_DIR': '/home/hadjik01/miniforge3/envs/tf/share/glib-2.0/schemas',
        'LOGNAME': 'hadjik01',
        'XDG_SESSION_TYPE': 'tty',
        'CONDA_PREFIX': '/home/hadjik01/miniforge3/envs/tf',
        'GSETTINGS_SCHEMA_DIR_CONDA_BACKUP': '',
        'MOTD_SHOWN': 'pam',
        'HOME': '/home/hadjik01',
        'LC_PAPER': 'de_DE.UTF-8',
        'LANG': 'en_US.UTF-8',
        'LS_COLORS': 'rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=

In [5]:

with h5py.File('/echo3/hadjik01/Paper Network Data/Heidelberg/Network/Folds/Fold 3/Trans/Validation/ValHeidelTransFold3.mat', 'r') as f:

    y_val  = f['lvSaveDataInput'][:,:,:,:]            
    x_val  = f['lvLovalizerSave'][:,:,:,:]            

x_val = np.moveaxis(x_val, 1, -1)
y_val = np.moveaxis(y_val, 1, -1)

print(x_val.shape)
print(y_val.shape)

(39, 96, 128, 65)
(39, 96, 128, 16)


In [6]:
with h5py.File('/echo3/hadjik01/Paper Network Data/Heidelberg/Network/Folds/Fold 3/Trans/Train/TrainHeidelTransFold3.mat', 'r') as f:
  
    y_train      = f['lvSaveDataInput'][:,:,:,:]
    x_train      = f['lvLovalizerSave'][:,:,:,:]
    
x_train = np.moveaxis(x_train, 1, -1)
y_train = np.moveaxis(y_train, 1, -1)

print(x_train.shape)
print(y_train.shape)   

(156, 96, 128, 65)
(156, 96, 128, 16)


In [71]:
input_shape  = x_train.shape[1:]   
output_shape = y_train.shape[1:]      


In [8]:
def symm_loss(target, prediction, eps=1e-8, l1factor=1.0, use_mask=False):
    """
    Symmetric loss for complex-valued tensors:
    total = P + l1factor * L1

    target, prediction: complex tensors of same shape
    returns: scalar loss (mean over all elements, masked if use_mask=True)
    """

    # cross term = |target.real * pred.imag - target.imag * pred.real|
    t_real = tf.math.real(target)
    t_imag = tf.math.imag(target)
    p_real = tf.math.real(prediction)
    p_imag = tf.math.imag(prediction)

    cross = tf.abs(t_real * p_imag - t_imag * p_real)  # real, >=0

    # |prediction|
    abs_pred = tf.abs(prediction)
    abs_target = tf.abs(target)

    # perpendicular component (P_raw)
    ploss_raw = cross / (abs_pred + eps)

    # angle < 90° ?  (Re(target / prediction) > 0)
    ratio = target / prediction
    angle_smaller_90 = tf.math.real(ratio) > 0  # boolean

    # symmetric perpendicular loss P
    ploss = tf.where(angle_smaller_90,
                     ploss_raw,
                     2.0 * abs_target - ploss_raw)

    # L1 component
    l1loss = tf.abs(prediction - target)

    # P + L1
    total = ploss + l1factor * l1loss

    if use_mask:
        mask = abs_target > 1e-3  # boolean
        mask = tf.cast(mask, total.dtype)
        masked_sum = tf.reduce_sum(total * mask)
        mask_count = tf.reduce_sum(mask)
        loss = masked_sum / (mask_count + eps)
    else:
        loss = tf.reduce_mean(total)

    return loss


In [45]:
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
       return optimizer.learning_rate# optimizer.lr
    return lr

def lr_scheduler(epoch, lr):
    decay_rate = 0.99944 
    decay_step = 1
    if epoch % decay_step == 0 and epoch:
        return lr * decay_rate
    return lr

class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Ensure logs is not None and contains the required keys
        if logs is None:
            logs = {}
        if "loss" in logs and "val_loss" in logs:
            print(
                "loss: {:.4e} - "
                "val_loss: {:.4e}".format(
                    logs["loss"], logs["val_loss"]
                )
            )
        else:
            print(f"Epoch {epoch} ended but logs are missing expected keys: {logs}")
            

In [61]:
init = RandomNormal(stddev=0.02)

def C2D_BLock(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    """
    Create a convolutional block with two Conv2D layers.
    
    Args:
        input_tensor: Input tensor from previous layer
        n_filters (int): Number of filters for convolution
        kernel_size (int): Size of convolution kernel (default: 3)
        batchnorm (bool): Whether to apply batch normalization (default: True)
    
    Returns:
        Output tensor after two convolution operations
    """
    # First convolution layer
    x = Conv2D(
        filters=n_filters, 
        kernel_size=(kernel_size, kernel_size), 
        kernel_initializer=init,
        padding="same"
    )(input_tensor)
    
    if batchnorm:
        x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    # Second convolution layer
    x = Conv2D(
        filters=n_filters, 
        kernel_size=(kernel_size, kernel_size), 
        kernel_initializer=init,
        padding="same"
    )(x)
    
    if batchnorm:
        x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    return x

In [72]:
def encoder(input_img, convolution_type, iterations, n_filters, dropout, batchnorm):
    """Simple encoder with configurable iterations."""
    features = []
    x = input_img
    
    for i in range(iterations):
        # Convolution block
        down = convolution_type(x, n_filters=n_filters*(2**i), kernel_size=3, batchnorm=batchnorm)
        features.append(down)
        
        # Max pooling (except for last iteration - that's the bottleneck)
        if i < iterations - 1:
            x = MaxPooling2D((2, 2))(down)
            # Dropout with increasing rate for deeper layers
            dropout_rate = dropout * (2 if i >= 2 else 1)
            x = Dropout(dropout_rate)(x)
        else:
            # Bottleneck dropout
            x = Dropout(dropout * 3)(down)
            features[-1] = x  # Update last feature with dropout
    
    return features


def decoder(bottleneck, skip_connections, convolution_type, transpose_conv_type, iterations, n_filters, dropout, batchnorm, heads=8):
    """Simple decoder with multiple heads."""
    outputs = []
    
    for head in range(heads):
        x = bottleneck
        
        for i in range(iterations):
            # Calculate filter size (decreasing: 8, 4, 2, 1)
            current_filters = n_filters * (2 ** (iterations - i - 1))
            
            # Upsampling with configurable transpose convolution
            x = transpose_conv_type(current_filters, (2, 2), strides=(2, 2), padding='same')(x)
            x = concatenate([x, skip_connections[i]])
            
            # Dropout (higher rate for first 2 iterations)
            dropout_rate = dropout * (2 if i < 2 else 1)
            x = Dropout(dropout_rate)(x)
            
            # Convolution block
            x = convolution_type(x, n_filters=current_filters, kernel_size=3, batchnorm=batchnorm)
        
        # Output layer
        output = Conv2D(2, (1, 1), activation='tanh', name=f"outputsCh{head+1}")(x)
        outputs.append(output)
    
    return outputs

def define_unet2D(input_img, n_filters=16, dropout=0.5, batchnorm=True):
    
    # --- Single-slice encoder (no multi-slice fusion) ---
    features = encoder(
        input_img=input_img,
        convolution_type=C2D_BLock,
        iterations=5,
        n_filters=n_filters,
        dropout=dropout,
        batchnorm=batchnorm,
    )
    
    # Unpack encoder features
    down1, down2, down3, down4, down5 = features   # normal UNet skip connections
    
    # --- Decoder with 8 heads ---
    outputs = decoder(
        bottleneck=down5,
        skip_connections=[down4, down3, down2, down1],
        convolution_type=C2D_BLock,
        transpose_conv_type=Conv2DTranspose,
        iterations=4,
        n_filters=n_filters,
        dropout=dropout,
        batchnorm=batchnorm,
        heads=8,
    )
    
    model = Model(inputs=[input_img], outputs=outputs)
    return model


In [73]:
from tensorflow.keras.callbacks import CSVLogger
LOG_DIR = "/echo3/hadjik01/ISMRM Cape Town Data/Result/Logger"          
csv_logger = CSVLogger(f"{LOG_DIR}/history.csv", append=True)



## checkpoint
filepath = "checks.weights.h5"

checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min', save_freq=100)
callbacks_list = [checkpoint, LearningRateScheduler(lr_scheduler),CustomCallback(), csv_logger]

lr_metric = get_lr_metric(optimizer)

input_img = Input(input_shape)
model = define_unet2D(input_img, n_filters=32, dropout=0.001, batchnorm=False)
optimizer = Adam(learning_rate=1e-04, beta_1=0.9, beta_2=0.999, epsilon=1e-07,clipnorm=1.0, amsgrad=False)

model.compile(optimizer=optimizer, loss='mean_squared_error',  metrics=['mse', lr_metric])
model.summary()                          #symm_loss_single


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_18 (InputLayer)       [(None, 96, 128, 65)]        0         []                            
                                                                                                  
 conv2d_8170 (Conv2D)        (None, 96, 128, 32)          18752     ['input_18[0][0]']            
                                                                                                  
 leaky_re_lu_8124 (LeakyReL  (None, 96, 128, 32)          0         ['conv2d_8170[0][0]']         
 U)                                                                                               
                                                                                                  
 conv2d_8171 (Conv2D)        (None, 96, 128, 32)          9248      ['leaky_re_lu_8124[0][0]

In [ ]:
# training of the model
history = model.fit(
    x=x_train,
    y={
        "outputsCh1": y_train[:, :, :,  0: 2],
        "outputsCh2": y_train[:, :, :,  2: 4],
        "outputsCh3": y_train[:, :, :,  4: 6],
        "outputsCh4": y_train[:, :, :,  6: 8],
        "outputsCh5": y_train[:, :, :,  8:10],
        "outputsCh6": y_train[:, :, :, 10:12],
        "outputsCh7": y_train[:, :, :, 12:14],
        "outputsCh8": y_train[:, :, :, 14:16]
    },
    validation_data=(
        x_val, 
        {
            "outputsCh1": y_val[:, :, :,  0: 2],
            "outputsCh2": y_val[:, :, :,  2: 4],
            "outputsCh3": y_val[:, :, :,  4: 6],
            "outputsCh4": y_val[:, :, :,  6: 8],
            "outputsCh5": y_val[:, :, :,  8:10],
            "outputsCh6": y_val[:, :, :, 10:12],
            "outputsCh7": y_val[:, :, :, 12:14],
            "outputsCh8": y_val[:, :, :, 14:16]
        }
    ),
    shuffle=True,
    epochs=2000,
    batch_size=1,
    callbacks=callbacks_list
)

Epoch 1/2000


2025-09-30 09:45:36.005429: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2025-09-30 09:45:39.047662: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8907
2025-09-30 09:45:41.055888: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f436d2c5c50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-09-30 09:45:41.055924: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro M4000, Compute Capability 5.2
2025-09-30 09:45:41.061348: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-09-30 09:45:41.207359: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster usi

 99/169 [================>.............] - ETA: 9s - loss: 0.0325 - outputsCh1_loss: 0.0028 - outputsCh2_loss: 0.0030 - outputsCh3_loss: 0.0034 - outputsCh4_loss: 0.0062 - outputsCh5_loss: 0.0054 - outputsCh6_loss: 0.0050 - outputsCh7_loss: 0.0037 - outputsCh8_loss: 0.0030 - outputsCh1_mse: 0.0028 - outputsCh1_lr: 1.0000e-04 - outputsCh2_mse: 0.0030 - outputsCh2_lr: 1.0000e-04 - outputsCh3_mse: 0.0034 - outputsCh3_lr: 1.0000e-04 - outputsCh4_mse: 0.0062 - outputsCh4_lr: 1.0000e-04 - outputsCh5_mse: 0.0054 - outputsCh5_lr: 1.0000e-04 - outputsCh6_mse: 0.0050 - outputsCh6_lr: 1.0000e-04 - outputsCh7_mse: 0.0037 - outputsCh7_lr: 1.0000e-04 - outputsCh8_mse: 0.0030 - outputsCh8_lr: 1.0000e-04
Epoch 1: loss improved from inf to 0.03238, saving model to checks.weights.h5
169/169 [==============================] - 74s 180ms/step - loss: 0.0306 - outputsCh1_loss: 0.0026 - outputsCh2_loss: 0.0028 - outputsCh3_loss: 0.0031 - outputsCh4_loss: 0.0058 - outputsCh5_loss: 0.0052 - outputsCh6_loss: 0.

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



 74/169 [============>.................] - ETA: 12s - loss: 0.0030 - outputsCh1_loss: 1.5838e-04 - outputsCh2_loss: 1.5326e-04 - outputsCh3_loss: 2.1447e-04 - outputsCh4_loss: 3.5415e-04 - outputsCh5_loss: 3.3671e-04 - outputsCh6_loss: 8.9298e-04 - outputsCh7_loss: 7.0751e-04 - outputsCh8_loss: 1.6031e-04 - outputsCh1_mse: 1.5838e-04 - outputsCh1_lr: 8.8158e-05 - outputsCh2_mse: 1.5326e-04 - outputsCh2_lr: 8.8158e-05 - outputsCh3_mse: 2.1447e-04 - outputsCh3_lr: 8.8158e-05 - outputsCh4_mse: 3.5415e-04 - outputsCh4_lr: 8.8158e-05 - outputsCh5_mse: 3.3671e-04 - outputsCh5_lr: 8.8158e-05 - outputsCh6_mse: 8.9298e-04 - outputsCh6_lr: 8.8158e-05 - outputsCh7_mse: 7.0751e-04 - outputsCh7_lr: 8.8158e-05 - outputsCh8_mse: 1.6031e-04 - outputsCh8_lr: 8.8158e-05
Epoch 226: loss did not improve from 0.00078
169/169 [==============================] - 23s 138ms/step - loss: 0.0026 - outputsCh1_loss: 2.4014e-04 - outputsCh2_loss: 2.4542e-04 - outputsCh3_loss: 2.2732e-04 - outputsCh4_loss: 3.4073e-04

In [25]:
print('end')

end


In [26]:
#save weights 

model.save_weights('/echo3/hadjik01/ISMRM Cape Town Data/Result/B1+Minne_Trans_Weights_DO.001_Dec0.99944_F32_E4000.h5', 'r')

In [27]:
#save history

np.save('/echo3/hadjik01/ISMRM Cape Town Data/Result/B1+Minne_Trans_History_DO0.001_Dec0.99944_F32_E4000NoBatch',history.history)

print(np.shape(history))

()


In [28]:
# load data for in vivo application

with h5py.File('/echo3/hadjik01/ISMRM Cape Town Data/ValData/TransHeidelFold1Val.mat', 'r') as f:
   y_test = f['lvSaveDataInput'][:,:,:,:]
   x_test = f['lvLovalizerSave'][:,:,:]
    
### ### ### ### ###

print("x_shape:", "", x_test.shape)
print("y_shape:", "", y_test.shape)

print("\n")
x = x_test = np.moveaxis(x_test, 1, -1)
y = y_test = np.moveaxis(y_test, 1, -1)
print("\n")

print("x_shape_new_axis:", "", x.shape)
print("y_shape_new_axis:", "", y.shape)

### ### ### ### ###

x_shape:  (39, 65, 96, 128)
y_shape:  (39, 16, 96, 128)




x_shape_new_axis:  (39, 96, 128, 65)
y_shape_new_axis:  (39, 96, 128, 16)


In [29]:
#do prediction

p = prediction = np.array(model.predict(x_test))


### ### ### ### ###


print("Prediction shape:","",p.shape)

print("\n")


if isinstance(p[...,0], complex):
  print ('prediction with 32 filters [...,0] is complex')

if isinstance(p[...,1], complex):
  print ('prediction with 32 filters [...,1] is complex')


### ### ### ### ###

2/2 [==============================] - 1s 377ms/step
Prediction shape:  (8, 39, 96, 128, 2)




In [30]:
#save complex truth

# array slicing -> leaves the frist 3 dimensions unchanged, and only takes every second element of the fourth dimension
ct = complex_true = complex_true = np.array(y_val[...,0::2] + 1j*y_val[...,1::2])

at = abs_true = np.moveaxis(np.abs(complex_true),-1,0)
pt = phase_true = np.moveaxis(np.angle(complex_true),-1,0)

np.save('/echo3/hadjik01/ISMRM Cape Town Data/Result/B1+Minne_Trans_ComplexTrue_DO0.001_Dec0.99944_F32_E4000NoBatch',complex_true)

In [31]:
#save complex prediction

cp =  complex_prediction = complex_prediction = np.array(p[...,0]) + 1j*np.array(p[...,1])

ap = abs_prediction = np.abs(complex_prediction)
pp = phase_prediction = np.angle(complex_prediction)

np.save('/echo3/hadjik01/ISMRM Cape Town Data/Result/B1+Minne_Trans_ComplexPrediction_DO0.001_Dec0.99944_F32_E4000NoBatch',complex_prediction)

In [32]:
def phase_diff(pp,pt):
  diff = pp/pt

  diff[np.isnan(diff)] = 0

  return diff

dp = phase_diff(pp,pt)

/tmp/ipykernel_3791413/3813304261.py:2: RuntimeWarning: divide by zero encountered in divide
  diff = pp/pt
/tmp/ipykernel_3791413/3813304261.py:2: RuntimeWarning: invalid value encountered in divide
  diff = pp/pt


In [33]:
### ### ### ### ###


print("abs_true.shape :", abs_true.shape)
print("abs_prediction.shape :", abs_prediction.shape)

print("\n")

print("phase_true.shape :", abs_true.shape)
print("phase_prediction.shape :", abs_prediction.shape)
### ### ### ### ###

abs_true.shape : (8, 39, 96, 128)
abs_prediction.shape : (8, 39, 96, 128)


phase_true.shape : (8, 39, 96, 128)
phase_prediction.shape : (8, 39, 96, 128)


In [34]:
print('end of script')

end of script
